In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark as ps
import pyspark.sql.types as types
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import to_timestamp

# Check into a recommender library called "surprise"

In [3]:
# review_df = pd.read_json('review.json')

In [4]:
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('lecture')
         .getOrCreate()
        )
sc = spark.sparkContext

review_df = spark.read.json("data/review.json")
# checkin_df = spark.read.json("data/checkin.json")
# tip_df = spark.read.json("data/tip.json")
user_df = spark.read.json("data/user.json")
business_df = spark.read.json("data/business.json")

In [5]:
# Displays the content of the DataFrame to stdout
# review_df.columns

In [6]:
# checkin_df.head()

In [7]:
# tip_df.head()

In [8]:
# photo_df.head()

In [9]:
# business_df.take(2)

In [10]:
# user_df.columns

# Basic Cleaning

In [ ]:
# # Check for null values
# frames = [business_df, review_df, user_df]

# for df in frames:
#     for c in df.columns:
#         null_value_count = df.where(df[c] == None).count()
#         print('Column: {}\nNull count: {}\n\n'.format(c, str(null_value_count)))
    
# # This is not showing any null values for the moment, remain skeptical, possible misuse of spark

# Basic EDA

In [ ]:
# Count of distinct values for all fields in review_df:
this = business_df.agg(*(countDistinct(col(c)).alias(c) for c in business_df.columns))
this.show(1)

In [ ]:
bsubset_df = business_df.filter((business_df.categories.like('%Restaurants%')) &
                                (business_df.state == 'AZ')                               
                               )
print('There are {} Arizona restaurants in the dataset.'.format(bsubset_df.count()))

## Major Metros in Dataset
#### Las Vegas
* Henderson
#### Toronto
* Missassauga
* Markham
* North York
#### Phoenix - 56,686
* Mesa
* Tempe
* Scottsdale
* Chandler
* Glendale
* Gilbert
* Peoria
* Surprise
#### Calgary
#### Pittsburgh
#### Montreal
#### Cleveland
#### Madison, WI
#### Champaign, IL

### Which region should we select for proceeding with this study? 

Let's look at the densities to decide:

In [ ]:
def calculate_region_density(state, businesses, bus_reviews):
    num_businesses = businesses.filter((businesses.categories.like('%Restaurants%')) &
                                (business_df.state == state)).count()
    num_reviews = bus_reviews.filter((bus_reviews.categories.like('%Restaurants%')) &
                                (bus_reviews.state == state)).count()
    return num_reviews/num_businesses

def compare_region_densities(regions, businesses, bus_reviews):
    max_density = 0
    density_dict = {}
    for region in regions:
        density = calculate_region_density(region, businesses, bus_reviews)
        density_dict[region] = density
        if density > max_density:
            max_density = density
            max_region = region

    print('Best Region: {}\nBest Density: {}'.format(max_region, max_density))
    return density_dict

In [ ]:
regions = ['AZ', 'ON', 'NC', 'AB', 'NV', 'OH', 'PA', 'IL', 'SC', 'QC']

density_dict = compare_region_densities(regions, business_df, bus_review_df)

pd.DataFrame.from_dict(density_dict, orient = 'index').sort_values(0, ascending = False).plot.bar(legend = False, figsize = (10, 6))
plt.ylabel('Review Density')
plt.xlabel('Region')
plt.title('Review Density by Region')

## The winner: Arizona

We are proceeding with Arizona based businesses for this case study. Although Nevada has a greater density of reviews/restaurant, this data scientist considers it to be an anomaly in the restaurant space as well as user space given the impact tourism would have on restaurant visits/reviews. Arizona has the next highest density, would be expected to include a healthy mix of resident and non-resident reviews, most reviews are concentrated around Phoenix, and we would expect Phoenix to have a diverse group of users.

# Query for reviews of restaurants in Arizona and then convert to Pandas

In [11]:
# Make "star" columns unique on business_df and review_df to avoid confusion
business_df = business_df.withColumnRenamed("stars","avg_stars")
review_df = review_df.withColumnRenamed("stars","review_stars")

In [12]:
# Filter business_df down to only businesses in Arizona
business_df = business_df.filter((business_df.state == 'AZ') &
                                (business_df.categories.like('%Restaurants%'))
                                )

In [13]:
# Join business and review df's 
bus_review_df = review_df.join(business_df, review_df.business_id == business_df.business_id, how='left')

In [1]:
# # Convert to pandas to make it easier to work
# bus_review_df = bus_review_df.select("*").toPandas()

# # Not Currently working on my machine, getting EC2 instance ready

# Further EDA

In [ ]:
# What is the distribution of ratings per restaurant and ratings per user?
ratings_per_restaurant = bus_review_df.groupBy()
